In [1]:
import os
import pandas as pd
import numpy as np


In [2]:
!git clone -b 18-pl_light_bolts_model_consume_data_changed https://github.com/nmud19/thesisGAN.git

Cloning into 'thesisGAN'...
remote: Enumerating objects: 512, done.
remote: Counting objects: 100% (258/258), done.
remote: Compressing objects: 100% (158/158), done.
remote: Total 512 (delta 157), reused 162 (delta 100), pack-reused 254
Receiving objects: 100% (512/512), 14.05 MiB | 6.21 MiB/s, done.
Resolving deltas: 100% (254/254), done.


In [3]:
os.chdir("thesisGAN")
from app.consume_data import consume_data

In [4]:
!pip install pytorch-lightning lightning-bolts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.8/316.8 kB 499.5 kB/s eta 0:00:00


In [5]:
from typing import Union, List

import torch
import torchvision
from pl_bolts.models.gans import Pix2Pix
from pytorch_lightning.utilities.types import EPOCH_OUTPUT

from pytorch_lightning.callbacks import EarlyStopping
from app.consume_data import consume_data
import pytorch_lightning as pl
from pl_bolts.models.gans.pix2pix.components import PatchGAN


class OverpoweredPix2Pix(Pix2Pix):

    def validation_step(self, batch, batch_idx):
        """ Validation step """
        real, condition = batch
        with torch.no_grad():

            loss = self._disc_step(real, condition)
            self.log("Valid PatchGAN Loss", loss)

            loss = self._gen_step(real, condition)
            self.log("Valid Generator Loss", loss)

        return {
            'sketch': condition,
            'colour': real
        }

    def validation_epoch_end(self, outputs: Union[EPOCH_OUTPUT, List[EPOCH_OUTPUT]]) -> None:
        sketch = outputs[0]['sketch']
        colour = outputs[0]['colour']
        with torch.no_grad():
            gen_coloured = self.gen(sketch)
        grid_image = torchvision.utils.make_grid(
            [
                sketch[0], colour[0], gen_coloured[0],
            ],
            normalize=True
        )
        self.logger.experiment.add_image(f'Image Grid {str(self.current_epoch)}', grid_image, self.current_epoch)
        

class PatchGanChanged(OverpoweredPix2Pix):    
    def __init__(self, in_channels, out_channels):
        super(PatchGanChanged,self).__init__(
            in_channels=in_channels, 
            out_channels=out_channels
        )
        self.patch_gan = self.get_dense_PatchGAN(self.patch_gan)
        
    @staticmethod
    def get_dense_PatchGAN(disc: PatchGAN) -> PatchGAN:
        """ Add final layer to gan """
        disc.final = torch.nn.Sequential(
            disc.final,
            torch.nn.Flatten(),
            torch.nn.Linear(16 * 16, 1),
        )
        return disc

In [6]:
# CHECK FINAL SIZE

x = torch.randn(1, 3, 256, 256)
y = torch.randn(1, 3, 256, 256)

#
model = PatchGanChanged(
    in_channels=3,
    out_channels=3,
)
model.patch_gan(x,y)

/opt/conda/lib/python3.7/site-packages/torch/distributed/_sharded_tensor/__init__.py:10: DeprecationWarning: torch.distributed._sharded_tensor will be deprecated, use torch.distributed._shard.sharded_tensor instead
  DeprecationWarning


tensor([[0.0216]], grad_fn=<AddmmBackward0>)

In [7]:

def test_lightning_model(model):
    """ Test model e2e """
    # data Module
    anime_sketch_data_module = consume_data.AnimeSketchDataModule(
        data_dir="/kaggle/input/anime-sketch-colorization-pair/data/",
        val_batch_size=16,
        train_batch_size=64
    )
    # Trainer
    # epoch_inference_callback = lit_model.EpochInference(valid_dataloader,use_gpu=False)
    # checkpoint_callback = pl.callbacks.model_checkpoint.ModelCheckpoint()
    logger = pl.loggers.TensorBoardLogger("tb_logs_v2", name="lightning_logs")
    """
    early_stop_callback = EarlyStopping(
        monitor="val_PatchGAN_loss",
        # min_delta=0.00,
        patience=5,
        verbose=True,
        mode="min"
    )
    """
    trainer = pl.Trainer(
        #fast_dev_run=True,
        max_epochs=100,
        logger=logger,
        callbacks=[
            #early_stop_callback,
            #     # epoch_inference_callback,
            #     # checkpoint_callback,
            pl.callbacks.TQDMProgressBar(refresh_rate=10)
        ],
        default_root_dir="chk",
        accelerator='gpu', 
        devices=1,
        # progress_bar_refresh_rate=1
    )
    trainer.fit(
        model=model,
        datamodule=anime_sketch_data_module,
        # ckpt_path="/Users/nimud/Downloads/thesisGAN_9/tb_logs/pix2pix_lightning_model/version_0/checkpoints/epoch=9-step=17780.ckpt"
    )
    print("complete!")


In [8]:
model = PatchGanChanged(
    in_channels=3,
    out_channels=3,
)
test_lightning_model(model=model)

The train test dataset lengths are :  14224 3545


/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:585: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.object,
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:637: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.bool,
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/tensor_util.py:108: DeprecationWarning: `np.object` is a deprecated ali

Sanity Checking: 0it [00:00, ?it/s]

/opt/conda/lib/python3.7/site-packages/torch/utils/tensorboard/summary.py:443: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  image = image.resize((scaled_width, scaled_height), Image.ANTIALIAS)


Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

complete!
